In [1]:
import pandas as pd
import numpy as np

In [4]:
Y = pd.read_csv("./acol_simulated_Y.txt", sep="\t", index_col=0)

In [9]:
Y

,Y_state,molecules,species,fraction_of_one
position,,,,
0,1,cholesterol,human,0.99536
1,1,cholesterol,mosquito,0.93924
2,1,cholesterol,fly,0.94026
3,1,cholesterol,worm,0.93976
4,1,cholesterol,bacteria,0.93986
...,...,...,...,...
235,0,oxford,cat,0.33256
236,0,oxford,fish,0.33112
237,0,oxford,bird,0.33228


In [6]:
Z = pd.read_csv("./acol_simulated_Z_species.txt", sep="\t")

In [ ]:
matrix = np.array([[0, 0], [0, 0]])
# for i, row in Y.iterrows():